### Feature Engineering
Create features to be used in conventional models. Features aim to
capture characteristics of sarcastic tweets so that models are able
to accurately classify tweets.

Several functions in this notebook where inspired by the work in
this repo: https://github.com/MirunaPislar/Sarcasm-Detection

In [ ]:
import emoji
from nltk import ngrams
from nltk.tokenize import TweetTokenizer
import itertools
import os
import re
import numpy as np
import pandas as pd
from jx_dev.utils_text_clf import utils_text_clf as utils
import cb_dev.vocab_helpers as helper
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

Apply RegEx to split terms into multiple words. Helper for hashtag features

In [ ]:
def camel_case_split(term):
    term = re.sub(r'([0-9]+)', r' \1', term)
    term = re.sub(r'(1st|2nd|3rd|4th|5th|6th|7th|8th|9th|0th)', r'\1 ', term)
    splits = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', term)
    return [s.group(0) for s in splits]

Split hashtags into individual terms

In [ ]:
def split_hashtags(hashtag, word_list, verbose=False):
    if verbose:
        print("Hashtag is %s" % hashtag)
    # Get rid of the hashtag
    if hashtag.startswith('#'):
        term = hashtag[1:]
    else:
        term = hashtag

    # If the hastag is already an existing word (a single word), return it
    if word_list is not None and term.lower() in word_list:
        return ['#' + term]
    # First, attempt splitting by CamelCase
    if term[1:] != term[1:].lower() and term[1:] != term[1:].upper():
        splits = camel_case_split(term)
    elif '#' in term:
        splits = term.split("#")
    elif len(term) > 27:
        if verbose:
            print("Hashtag %s is too big so let as it is." % term)
        splits = [term]
    else:
        # Second, build possible splits and choose the best split by assigning
        # a "score" to each possible split, based on the frequency with which a word is occurring
        penalty = -69971
        max_coverage = penalty
        max_splits = 6
        n_splits = 0
        term = re.sub(r'([0-9]+)', r' \1', term)
        term = re.sub(r'(1st|2nd|3rd|4th|5th|6th|7th|8th|9th|0th)', r'\1 ', term)
        term = re.sub(r'([A-Z][^A-Z ]+)', r' \1', term.strip())
        term = re.sub(r'([A-Z]{2,})+', r' \1', term)
        splits = term.strip().split(' ')
        if len(splits) < 3:
            # Splitting lower case and uppercase hashtags in up to 5 words
            chars = [c for c in term.lower()]
            found_all_words = False

            while n_splits < max_splits and not found_all_words:
                for index in itertools.combinations(range(0, len(chars)), n_splits):
                    output = np.split(chars, index)
                    line = [''.join(o) for o in output]
                    score = 0.0
                    for word in line:
                        stripped = word.strip()
                        if stripped in word_list:
                            score += int(word_list.get(stripped))
                        else:
                            if stripped.isnumeric():  # not stripped.isalpha():
                                score += 0.0
                            else:
                                score += penalty
                    score = score / float(len(line))
                    if score > max_coverage:
                        splits = line
                        max_coverage = score
                        line_is_valid_word = [word.strip() in word_list if not word.isnumeric()
                                              else True for word in line]
                        if all(line_is_valid_word):
                            found_all_words = True
                n_splits = n_splits + 1
    splits = ['#' + str(s) for s in splits]
    if verbose:
        print("Split to: ", splits)
    return splits

Initial tweet cleaning - useful to filter data before tokenization

In [ ]:
def clean_tweet(tweet, word_list, split_hashtag_method=split_hashtags, replace_user_mentions=True,
                remove_hashtags=False, remove_emojis=False, all_to_lower_case=False):
    # Add white space before every punctuation sign so that we can split around it and keep it
    tweet = re.sub('([!?*&%"~`^+{}])', r' \1 ', tweet)
    tweet = re.sub('\s{2,}', ' ', tweet)
    tokens = tweet.split()
    valid_tokens = []
    for word in tokens:
        # Never include #sarca* hashtags
        if word.lower().startswith('#sarca'):
            continue
        # Never include URLs
        if 'http' in word:
            continue
        # Replace specific user mentions with a general user name
        if replace_user_mentions and word.startswith('@'):
            word = '@user'
        # Split or remove hashtags
        if word.startswith('#'):
            if remove_hashtags:
                continue
            splits = split_hashtag_method(word[1:], word_list)
            if all_to_lower_case:
                valid_tokens.extend([split.lower() for split in splits])
            else:
                valid_tokens.extend(splits)
            continue
        if remove_emojis and word in emoji.UNICODE_EMOJI:
            continue
        if all_to_lower_case:
            word = word.lower()
        valid_tokens.append(word)
    return ' '.join(valid_tokens)

Helper function for creating ngram features

In [ ]:
def get_ngram_list(tokens, n):
    tokens = [t for t in tokens if not t.startswith('#')]
    tokens = [t for t in tokens if not t.startswith('@')]
    ngram_list = [gram for gram in ngrams(tokens, n)]
    return ngram_list

Helper function for creating ngram features

In [ ]:
def get_ngram_features(ngrams, ngram_map):
    feature_list = [0] * np.zeros(len(ngram_map))
    for gram in ngrams:
        if gram in ngram_map:
            feature_list[ngram_map[gram]] += 1.0
    return feature_list

Helper function for creating ngram features

In [ ]:
def get_ngrams(tokens, n, stopwords):
    if len(n) < 1:
        return {}
    filtered = []
    for t in tokens:
        if t not in stopwords and t.isalnum():
            filtered.append(t)
    tokens = filtered
    ngram_features = {}
    for i in n:
        ngram_list = [gram for gram in ngrams(tokens, i)]
        ngram_features[i] = ngram_list
    return np.array(ngram_features)

Helper function for creating ngram features

In [ ]:
def create_ngram_columns(df):
    df["unigrams"] = df["tokens"].apply(get_ngram_list, args=[1])
    df["bigrams"] = df["tokens"].apply(get_ngram_list, args=[2])
    df["trigrams"] = df["tokens"].apply(get_ngram_list, args=[3])

    return df


Helper function for creating ngram features

In [ ]:
def final_ngram_cols(df, ngram_map):
    df["unigram_features"] = df["unigrams"].apply(get_ngram_features, args=[ngram_map])
    df["bigram_features"] = df["bigrams"].apply(get_ngram_features, args=[ngram_map])
    df["trigram_features"] = df["trigrams"].apply(get_ngram_features, args=[ngram_map])
    df["ngram_features"] = df["unigram_features"] + df["bigram_features"] + df["trigram_features"]

    ngram_cols = ["n-gram-" + str(i) for i in range(len(ngram_map))]
    df[ngram_cols] = pd.DataFrame(df.ngram_features.tolist(), index=df.index)
    df["ngram_feature"] = df[ngram_cols].sum(axis=1)
    df.drop(columns=ngram_cols, inplace=True)

    return df

Create features based on tweet characteristics such as number of users tagged

In [ ]:
def get_simple_features(df, emoji_sent_dict):
    tokenizer = TweetTokenizer(preserve_case=True, reduce_len=False, strip_handles=True)
    df["users_tagged"] = df.response.str.count("@USER")
    df["response"] = df.response.str.replace('@USER', '').str.strip()
    df["tokens"] = df["response"].str.split()

    df["num_hashtags"] = df.response.str.count("#")
    df["num_capital"] = df.response.str.findall(r'[A-Z]').str.len()
    # Currently includes punctuation
    df["tweet_length_words"] = df.tokens.str.len()
    df["tweet_length_char"] = df.tokens.apply(lambda tweet: sum([len(w) for w in tweet]))
    df["average_token_length"] = df["tweet_length_words"].astype(float) / df["tweet_length_char"]

    df["contains_laughter"] = df.tokens.apply(
        lambda tweet: len([1 for w in tweet if (w.lower().startswith("haha")) or (re.match('l(o)+l$', w.lower()))])
    )
    df["contains_ellipses"] = df.tokens.apply(lambda tweet: len([w for w in tweet if (w == '...') | (w == '..')]))
    df["strong_negations"] = df.tokens.apply(lambda tweet: len([w for w in tweet if w in helper.strong_negations]))
    df["strong_affirmatives"] = df.tokens.apply(lambda tweet: len([w for w in tweet if w in helper.strong_affirmatives]))
    df["interjections"] = df.tokens.apply(lambda tweet: len([w for w in tweet if w in helper.interjections]))
    df["intensifiers"] = df.tokens.apply(lambda tweet: len([w for w in tweet if w in helper.intensifiers]))
    df["punctuation"] = df.tokens.apply(lambda tweet: len([w for w in tweet if w in helper.punctuation]))
    df["emojis"] = df.tokens.apply(lambda tweet: len([w for w in tweet if w in emoji.UNICODE_EMOJI]))
    df['negative emoji'] = df.tokens.apply(lambda tweet: sum([float(emoji_sent_dict[t][0]) for t in tweet if t in emoji_sent_dict.keys()]))
    df['neutral emoji'] = df.tokens.apply(lambda tweet: sum([float(emoji_sent_dict[t][1]) for t in tweet if t in emoji_sent_dict.keys()]))
    df['positive emoji'] = df.tokens.apply(lambda tweet: sum([float(emoji_sent_dict[t][2]) for t in tweet if t in emoji_sent_dict.keys()]))

    # Clean up responses for tokenizing
    df["tokens"] = df["response"].apply(tokenizer.tokenize)
    return df

Apply similar logic to each tweet in the context of a response

In [ ]:
def context_features(df, emoji_sent_dict):
    tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
    users_tagged = 0
    tokens = []

    num_hashtags = 0
    num_capital = 0
    # Currently includes punctuation
    tweet_length_words = 0
    tweet_length_char = 0
    average_token_length = 0

    contains_laughter = 0
    contains_ellipses = 0
    strong_negations = 0
    strong_affirmatives = 0
    interjections = 0
    intensifiers = 0
    punctuation = 0
    emojis = 0
    negative_emoji = 0
    neutral_emoji = 0
    positive_emoji = 0

    for tweet in df.loc['context']:
        users_tagged += tweet.count("@USER")
        tokens.append(tokenizer.tokenize(tweet))

        num_hashtags += tweet.count("#")
        num_capital += tweet.count(r'[A-Z]')
        # Currently includes punctuation
        tweet_length_words += len(tweet.split())
        tweet_length_char += len(tweet)
        average_token_length += len(tweet.split())/len(tweet)
        for w in tweet:
            if (w.lower().startswith("haha")) or (re.match('l(o)+l$', w.lower())):
                contains_laughter += 1

        contains_ellipses += len([w for w in tweet if (w == '...') | (w == '..')])
        strong_negations += len([w for w in tweet if w in helper.strong_negations])
        strong_affirmatives += len([w for w in tweet if w in helper.strong_affirmatives])
        interjections += len([w for w in tweet if w in helper.interjections])
        intensifiers += len([w for w in tweet if w in helper.intensifiers])
        punctuation += len([w for w in tweet if w in helper.punctuation])
        emojis += len([w for w in tweet if w in emoji.UNICODE_EMOJI])
        negative_emoji += sum([float(emoji_sent_dict[t][0]) for t in tweet if t in emoji_sent_dict.keys()])
        neutral_emoji += sum([float(emoji_sent_dict[t][1]) for t in tweet if t in emoji_sent_dict.keys()])
        positive_emoji += sum([float(emoji_sent_dict[t][2]) for t in tweet if t in emoji_sent_dict.keys()])
    df["context_users_tagged"] = users_tagged
    df["context_tokens"] = tokens
    df["context_num_hashtags"] = num_hashtags
    df["context_num_capital"] = num_capital
    df["context_tweet_length_words"] = tweet_length_words
    df["context_tweet_length_char"] = tweet_length_char
    df["context_average_token_length"] = average_token_length
    df["context_contains_laughter"] = contains_laughter
    df["context_contains_ellipses"] = contains_ellipses
    df["context_strong_negations"] = strong_negations
    df["context_strong_affirmatives"] = strong_affirmatives
    df["context_interjections"] = interjections
    df["context_intensifiers"] = intensifiers
    df["context_punctuation"] = punctuation
    df["context_emojis"] = emojis
    df['context_negative emoji'] = negative_emoji
    df['context_neutral emoji'] = neutral_emoji
    df['context_positive emoji'] = positive_emoji

    return df

Load list of stopwords

In [ ]:
stopword_list = []

with open("stopwords.txt", 'r') as file:
    text = file.read()
    file.close()
    stopword_list = text.split("\n")

Load dictionary of words with associated popularity values

In [ ]:
word_list = {}

with open("word_list.txt", 'r') as file:
    lines = file.read()
    file.close()
    for line in lines.split("\n"):
        key, value = line.split("\t")
        word_list[key] = value

Load list of emojis with sentiment information

In [ ]:
emoji_sent_dict = {}
with open("emoji_sent_dict.txt", 'r') as file:
    lines = file.read()
    file.close()
    for line in lines.split("\n"):
        key = line.split("\t")[0]
        value = line.split("\t")[1:]
        emoji_sent_dict[key] = value

Load train data

In [ ]:
data_dir = os.path.dirname(os.getcwd()) + '/data'
filename = 'train.jsonl'
file = os.path.join(data_dir, filename)  # .json file
train_df = utils.parse_json(file)

Load test data

In [ ]:
test_filename = 'test.jsonl'
test_file = os.path.join(data_dir, test_filename)  # .json file
test_df = utils.parse_json(test_file)

Clean tweets

In [ ]:
train_df["response"] = train_df.response.apply(clean_tweet, args=[word_list])
test_df["response"] = test_df.response.apply(clean_tweet, args=[word_list])

Get simple featues for response tweets

In [ ]:
train_df = get_simple_features(train_df, emoji_sent_dict)
test_df = get_simple_features(test_df, emoji_sent_dict)

Create a column for each unigram, bigram and trigram that occurs in the train data. Same number of time each
n-gram occurs in the data

In [ ]:
unigrams = Counter()
bigrams = Counter()
trigrams = Counter()
train_df = create_ngram_columns(train_df)
test_df = create_ngram_columns(test_df)

Update counters for each n-gram

In [ ]:
train_df["unigrams"].apply(unigrams.update)
train_df["bigrams"].apply(bigrams.update)
train_df["trigrams"].apply(trigrams.update)
print(train_df.head())

Keep n-grams that occur more than once in data

In [ ]:
unigram_tokens = [k for k, c in unigrams.items() if c > 1]
bigram_tokens = [k for k, c in bigrams.items() if c > 1]
trigram_tokens = [k for k, c in trigrams.items() if c > 1]

Create dictionary of n-grams that occur more than once

In [ ]:
ngram_map = dict()
all_ngrams = unigram_tokens
all_ngrams.extend(bigram_tokens)
all_ngrams.extend(trigram_tokens)
for i in range(0, len(all_ngrams)):
    ngram_map[all_ngrams[i]] = i

Keep n-gram columns for n-grams that occur most often then add all n-grams columns together for one feature

In [ ]:
train_df = final_ngram_cols(train_df, ngram_map)
test_df = final_ngram_cols(test_df, ngram_map)

Get context features for each tweet

In [ ]:
train_df = train_df.apply(context_features, axis=1, args=[emoji_sent_dict])
test_df = test_df.apply(context_features, axis=1, args=[emoji_sent_dict])

Drop unneeded columns

In [ ]:
cols_to_drop = [
    "context",
    "tokens",
    "response",
    "unigrams",
    "bigrams",
    "trigrams",
    "unigram_features",
    "bigram_features",
    "trigram_features",
    "ngram_features",
    "context_tokens"
]
train_df.drop(columns=cols_to_drop, inplace=True)
test_df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
print(train_df.dtypes)
print(test_df.dtypes)

In [ ]:
train_df.fillna(0, inplace=True)
test_df.fillna(0, inplace=True)

In [ ]:
print(train_df.columns)
print(test_df.columns)

Save data

In [ ]:
train_df.to_csv("data/train_feature_engineering.csv", index=False)
test_df.to_csv("data/test_feature_engineering.csv", index=False)